# Matmul Dataflow

* 这个例子中假设我们的设备信息是：
  * (a) 处理器上支持的矩阵乘指令是 _8x16x8_
  * (b) ___[K]___ L1 tile 需要循环 5x5 次 KernelFunctionCall 
  * (c) ___[S]___ L2 tile 需要循环 4x4 次 DMA C2S
  * (d) ___[C]___ L3 tile 需要循环 3x3 次 DMA D2C
  * (e) ___[P]___ L3 tile view：无论是 _4C_ 还是 _1C_ 都不需要进行 SliceD2D
    * _4C_ 的时候需要在算子进行计算之前把每个C的亲和 Tensor准备好，不需要将这个逻辑集成到每一个算子中。
    * _1C_ 的时候比如有4个SIP，那么这里只是确定每个SIP的负载，只是一个View而不需要真正的切分。


* 图示

```
假设每个Cluster有4个SIP，这里是 _1C_ 分到的计算:
--------------------------------------------------------------------------------
Global:
  Global-M = PM x 2 = 960
  Global-N = PN x 2 = 1920
  Global-K = PK x 1 = 160
            ________________________________________________
           |                        |                       |
           |________________________|_______________________|
                                 B[K, N]
      Global(L3)
    __      ________________________________________________ 
   |  |    |PX-L3                   |PX-L3                  |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |__|    |________________________|_______________________|
   |  |    |PX-L3                   |PX-L3                  |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |  |    |                        |                       |
   |__|    |________________________|_______________________|
  A[M,K]                        C[K, N]


--------------------------------------------------------------------------------
L3:
  PM = CM x 3 = 480
  PN = CN x 3 = 960
  PK = CK x 1 = 160
            __________________________
           |        |        |        |
           |________|________|________|
                   [PK, PN]

    __      __________________________
   |  |    |CX-L2   |        |        |
   |__|    |________|________|________|
   |  |    |        |        |        |
   |__|    |________|________|________|
   |  |    |        |        |        |
   |__|    |________|________|________|
[PM, PK]            [PM, PN]


--------------------------------------------------------------------------------
L2:（放大图像）
  CM = SM x 4 = 160
  CN = SN x 4 = 320
  CK = SK x 4 = 160
                  ___________________
                 |_SN_|_SN_|_SN_|_SN_| SK0
                 |____|____|____|____| SK1
                 |____|____|____|____| SK2
                 |____|____|____|____| SK3
                       [CK, CN]
              (CX-L2)
 ____________     ___________________
|SK|SK|SK|SK|    |_SX_|____|____|____| SM0
|__|__|__|__|    |____|____|____|____| SM1
|__|__|__|__|    |____|_L1_|____|____| SM2
|__|__|__|__|    |____|____|____|____| SM3
  [CM,CK]              [CM, CN]


--------------------------------------------------------------------------------
L1:（放大图像）
  SM = KM * 5 = 40
  SN = KN * 5 = 80
  SK = KK * 5 = 40
                     ________________________
                    | KK0|    |    |    |    |
                    | KK1|    |    |    |    |
                    | KK2|    |    |    |    |
                    | KK3|    |    |    |    |
                    |_KK4|____|____|____|____|
                             [SK, CN]
           SX-L1
 ______________      ________________________
|KK|KK|KK|KK|KK|    |_KN0|_KN1|_KN2|_KN3|_KN4| KM0
|  |  |  |  |  |    |____|____|____|____|____| KM1
|  |  |  |  |  |    |____|____|_L0_|____|____| KM2
|  |  |  |  |  |    |____|____|____|____|____| KM3
|__|__|__|__|__|    |____|____|____|____|____| KM4
    [SM, SK]                  [SM, SN]



--------------------------------------------------------------------------------
Instruction: GEMM_8_16_8:

   c[8, 16] = a[8, 8] * b[8, 16] + c[8, 16]


```




## Base Matmul

In [ ]:
import tvm
from tvm import te 

# ---------------
# op config
# ---------------

# dtu.instruction
KM = 8
KN = 16
KK = 8

# dtu.sdb (tile) = local
SM = KM * 5
SN = KN * 5
SK = KK * 5

# dtu.csb (tile) = global/shared (use global) = CX
CM = SM * 4
CN = SN * 4
CK = SK * 4

# dtu.sip (tile) = processor = PX = tvm.parallel
PM = CM * 3
PN = CN * 3
PK = CK

# feature map
M = PM * 2
N = PN * 2
K = PK

print('*'*64)
print('X: {}'.format([M, N, K]))
print('PX:{}'.format([PM, PN, PK]))
print('CX:{}'.format([CM, CN, CK]))
print('SX:{}'.format([SM, SN, SK]))
print('KX:{}'.format([KM, KN, KK]))
print('*'*64 + '\n')

# ---------------
# define compute
# ---------------
# define a reduce axis
k = te.reduce_axis((0, K), "k") 

# input tensors
l = te.placeholder((M, K), name="l")
r = te.placeholder((K, N), name="r")
# compute
o = te.compute((M, N), lambda m, n: te.sum(l[m, k] * r[k, n], axis=k), name="o")

# ---------------
# schedule op
# ---------------
# create a schedule
s = te.create_schedule(o.op)
print(tvm.lower(s, [l, r, o], simple_mode=True))



## Add New Storage

* `cache_read`: This will mutate the body of the readers. A new cache stage will be created for the tensor. ___Call this before doing any split/fuse schedule___.

In [ ]:

# add d2c
# l_l2 = s.cache_read(l, "global", [o])
# r_l2 = s.cache_read(r, "global", [o])
l_l2 = s.cache_read(l, "shared", [o])
r_l2 = s.cache_read(r, "shared", [o])

# add c2s
l_l1 = s.cache_read(l_l2, "local", [o])
r_l1 = s.cache_read(r_l2, "local", [o])

# add c2d
o_l2 = s.cache_write(o, "shared")

# add s2c
o_l1 = s.cache_write(o_l2, "local")

print(tvm.lower(s, [l, r, o], simple_mode=True))

## Tile output and insert output-L2 to output-L3

_PM_，_PN_，_CM_，_CN_ 只是对 output 进行 tiling 的参数，它们并不能描述其具体对应到什么存储层级、并行层级。下面这段代码进行了两个主要的操作：
* 对 `s[o]` 节点进行两次切分，生成 `pm`，`pn`，`cm`，`cn`
* 将 `s[o_l2]` 合并到 `s[o]`的 `pn`，这会触发 pass 对 `s[o_l2]` 做上面同样的两次切分。
* output tlie x2

In [ ]:

# axis
m, n = o.op.axis
# tile output firstly
m, n, pm, pn = s[o].tile(m, n, PM, PN)
# tile output secondly
pm, pn, cm, cn = s[o].tile(pm, pn, CM, CN)

s[o_l2].compute_at(s[o], pn)
print(tvm.lower(s, [l, r, o], simple_mode=True))

## 

## Tile output-L2 and insert output-L1 to output-L2

上面的操作是通过对`s[o]`的切分然后 `s[o_l2]` 合并到其中完成了同样的切分，下面的操作是在这个基础上，对`s[o_l2]`再进行一次切分，然后将`s[o_l1]`合并到`s[l_l2]`中同样会自动完成`s[o_l1]`的切分。这里的操作不会对 `s[o]` 产生影响， 从而实现了不同的切分。

* * output tlie x3


In [ ]:
cm, cn = o_l2.op.axis
print(cm)
print(cn)
cm, cn, sm, sn = s[o_l2].tile(cm, cn, SM, SN)

# insert tensors
s[o_l1].compute_at(s[o_l2], cn)
print(tvm.lower(s, [l, r, o], simple_mode=True))


## Tile output-L0

对`s[o_l1]`进行进一步切分，并且将 `s[l_l1]`, `s[l_l2]`, `s[r_l1]` 以及 `s[r_2]` 合并到 `s[o_l1]` 中。

* output tlie x4

In [ ]:

# tile - sdb & kernel - o_l1
sm, sn = o_l1.op.axis
print(sm)
print(sn)

sm, sn, km, kn = s[o_l1].tile(sm, sn, KM, KN)

# k
pk, ck = s[o_l1].split(k, CK)
ck, sk = s[o_l1].split(ck, SK)
sk, kk = s[o_l1].split(sk, KK)
# reorder 
s[o_l1].reorder(pk, ck, sm, sn, sk, km, kn, kk)

# insert tensors
s[l_l1].compute_at(s[o_l1], ck)
s[r_l1].compute_at(s[o_l1], ck)
s[l_l2].compute_at(s[o_l1], pk)
s[r_l2].compute_at(s[o_l1], pk)
print(tvm.lower(s, [l, r, o], simple_mode=True))


## Parallelize

In [ ]:

print('*'*64)
print('X: {}'.format([M, N, K]))
print('PX:{}'.format([PM, PN, PK]))
print('CX:{}'.format([CM, CN, CK]))
print('SX:{}'.format([SM, SN, SK]))
print('KX:{}'.format([KM, KN, KK]))
print('*'*64 + '\n')

# parallelize
s[o].parallel(m)
s[o].parallel(n)
print(tvm.lower(s, [l, r, o], simple_mode=True))
